In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(); sns.set_style('dark')

import os
import datetime

from sklearn.model_selection import train_test_split
import tensorflow as tf
import json

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-01-16 14:20:50.904560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737037250.940991  178357 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737037250.952199  178357 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-16 14:20:51.002067: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open("../configs/config.json", "r") as file:
    config = json.load(file)
    
IMAGE_SIZE = (config['model']['input_size'], config['model']['input_size'])
BATCH_SIZE = config['model']['batch_size']
NUM_CLASSES = config['model']['output_size']
INPUT_DIR = config['data']['processed_path']
CHECKPOINT_DIR = config['training']['checkpoint_dir']
FINAL_MODEL_DIR = config['training']['final_model_dir']

In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [4]:
model = tf.keras.models.load_model(FINAL_MODEL_DIR + "model15.01MobileNetV2.h5")

I0000 00:00:1737037254.551727  178357 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:05:00.0, compute capability: 8.6


In [5]:
for layer in model.layers:
    layer.trainable = True

In [6]:
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(CHECKPOINT_DIR + 'best_model.h5', monitor='val_loss', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir="logs/fit", histogram_freq=1)

In [8]:
# Створення генераторів даних
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Нормалізація пікселів до [0, 1]
    rotation_range=20,  # Випадкове обертання
    width_shift_range=0.2,  # Випадковий зсув по горизонталі
    height_shift_range=0.2,  # Випадковий зсув по вертикалі
    shear_range=0.2,  # Випадкове зсування
    zoom_range=0.2,  # Випадкове збільшення
    horizontal_flip=True,  # Випадкове віддзеркалення по горизонталі
    fill_mode="nearest",  # Заповнення відсутніх пікселів
    validation_split=0.2  # Розділення даних на навчання/валідацію
)

train_generator = train_datagen.flow_from_directory(
    INPUT_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode="rgb",  # Використовуємо RGB
    subset="training"  # Для навчання
)

val_generator = train_datagen.flow_from_directory(
    INPUT_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode="rgb",  # Використовуємо RGB
    subset="validation"  # Для валідації
)
test_generator = train_datagen.flow_from_directory(
    INPUT_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode="rgb",  # Використовуємо RGB
    subset="validation"  # Для валідації
)

Found 16000 images belonging to 250 classes.
Found 3999 images belonging to 250 classes.
Found 3999 images belonging to 250 classes.


In [9]:
history = model.fit(
    train_generator,  # Your training data generator
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of steps per epoch
    validation_data=val_generator,  # Your validation data generator
    validation_steps=val_generator.samples // val_generator.batch_size,  # Number of validation steps
    epochs=20,  # Train for more epochs
    callbacks = [reduce_lr, early_stopping, checkpoint]
)

/home/qizixxx/mlbook/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1737037282.075807  179020 service.cc:148] XLA service 0x7f0148003d00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737037282.075852  179020 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2025-01-16 14:21:22.514474: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737037285.634615  179020 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-16 14:21:28.494180: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5480', 228 bytes spill stores, 228 bytes spill loads

2025-01-16 14:21:28.686970: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5480_0', 4

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 937ms/step - accuracy: 0.4056 - loss: 2.4092

2025-01-16 14:25:58.678790: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1188', 4 bytes spill stores, 4 bytes spill loads

2025-01-16 14:25:58.819750: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1188', 100 bytes spill stores, 100 bytes spill loads

2025-01-16 14:25:59.257243: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1188', 444 bytes spill stores, 444 bytes spill loads



250/250 ━━━━━━━━━━━━━━━━━━━━ 357s 1s/step - accuracy: 0.4057 - loss: 2.4083 - val_accuracy: 0.5481 - val_loss: 1.7903 - learning_rate: 1.0000e-05
Epoch 2/20
250/250 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.5179 - loss: 1.8695 - val_accuracy: 0.5413 - val_loss: 1.8023 - learning_rate: 1.0000e-05
Epoch 3/20
 16/250 ━━━━━━━━━━━━━━━━━━━━ 3:32 907ms/step - accuracy: 0.5704 - loss: 1.6790

KeyboardInterrupt: 

In [ ]:
val_loss, val_accuracy = model.evaluate(test_generator)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')